Notebook for uploading PDF, extracting all Text and Pre-Processing using a 1B or 3B model

In [1]:
!pip install PyPDF2

In [14]:
pdf_path = './2402.13116v3.pdf'
DEFAULT_MODEL = "meta-llama/Llama-3.2-1B-Instruct"
#DEFAULT_MODEL = "meta-llama/Llama-3.2-1B-Instruct" <- Don't think this would be necessary

In [31]:
from difflib import HtmlDiff
from IPython.display import HTML, display

In [18]:
# Import necessary libraries
import PyPDF2
from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.notebook import tqdm
import warnings

accelerator = Accelerator()
device = accelerator.device

warnings.filterwarnings('ignore')

In [9]:
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: File not found at path: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: File is not a PDF")
        return False
    return True

In [10]:
def extract_text_from_pdf(file_path: str, max_chars: int = 100000) -> Optional[str]:
    if not validate_pdf(file_path):
        return None
    
    try:
        with open(file_path, 'rb') as file:
            # Create PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get total number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Processing PDF with {num_pages} pages...")
            
            extracted_text = []
            total_chars = 0
            
            # Iterate through all pages
            for page_num in range(num_pages):
                # Extract text from page
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Check if adding this page's text would exceed the limit
                if total_chars + len(text) > max_chars:
                    # Only add text up to the limit
                    remaining_chars = max_chars - total_chars
                    extracted_text.append(text[:remaining_chars])
                    print(f"Reached {max_chars} character limit at page {page_num + 1}")
                    break
                
                extracted_text.append(text)
                total_chars += len(text)
                print(f"Processed page {page_num + 1}/{num_pages}")
            
            final_text = '\n'.join(extracted_text)
            print(f"\nExtraction complete! Total characters: {len(final_text)}")
            return final_text
            
    except PyPDF2.PdfReadError:
        print("Error: Invalid or corrupted PDF file")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None


In [11]:
# Get PDF metadata
def get_pdf_metadata(file_path: str) -> Optional[dict]:
    if not validate_pdf(file_path):
        return None
    
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            metadata = {
                'num_pages': len(pdf_reader.pages),
                'metadata': pdf_reader.metadata
            }
            return metadata
    except Exception as e:
        print(f"Error extracting metadata: {str(e)}")
        return None

In [12]:
# Extract metadata first
print("Extracting metadata...")
metadata = get_pdf_metadata(pdf_path)
if metadata:
    print("\nPDF Metadata:")
    print(f"Number of pages: {metadata['num_pages']}")
    print("Document info:")
    for key, value in metadata['metadata'].items():
        print(f"{key}: {value}")

# Extract text
print("\nExtracting text...")
extracted_text = extract_text_from_pdf(pdf_path)

# Display first 500 characters of extracted text as preview
if extracted_text:
    print("\nPreview of extracted text (first 500 characters):")
    print("-" * 50)
    print(extracted_text[:500])
    print("-" * 50)
    print(f"\nTotal characters extracted: {len(extracted_text)}")

# Optional: Save the extracted text to a file
if extracted_text:
    output_file = 'extracted_text.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(extracted_text)
    print(f"\nExtracted text has been saved to {output_file}")

Extracting metadata...

PDF Metadata:
Number of pages: 44
Document info:
/Author: 
/CreationDate: D:20240311015030Z
/Creator: LaTeX with hyperref
/Keywords: 
/ModDate: D:20240311015030Z
/PTEX.Fullbanner: This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5
/Producer: pdfTeX-1.40.25
/Subject: 
/Title: 
/Trapped: /False

Extracting text...
Processing PDF with 44 pages...
Processed page 1/44
Processed page 2/44
Processed page 3/44
Processed page 4/44
Processed page 5/44
Processed page 6/44
Processed page 7/44
Processed page 8/44
Processed page 9/44
Processed page 10/44
Processed page 11/44
Processed page 12/44
Processed page 13/44
Processed page 14/44
Processed page 15/44
Processed page 16/44
Reached 100000 character limit at page 17

Extraction complete! Total characters: 100016

Preview of extracted text (first 500 characters):
--------------------------------------------------
1
A Survey on Knowledge Distillation of Large
Language Models
Xiaohan Xu1, M

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

SYS_PROMPT = """
You are a world class text pre-processor, here is the raw data from a PDF, please parse and return it in a way that is crispy and usable to send to a podcast writer.

The raw data is messed up with new lines, Latex math and you will see fluff that we can remove completely. Basically take away any details that you think might be useless in a podcast author's transcript.

Remember, the podcast could be on any topic whatsoever so the issues listed above are not exhaustive

The goal is to use this in a podcast research transcript so a lot of the emails, citations, and things like that can be removed-please be smart with what you remove and be creative ok?

Remember DO NOT START SUMMARIZING THIS, YOU ARE ONLY CLEANING UP THE TEXT AND RETURNING AS IS

Be very smart and aggressive with removing details, you will get a running portion of the text and keep returning the processed text.

ALWAYS start your response directly with processed text and NO ACKNOWLEDGEMENTS about my questions ok?
Here is the text:
"""

In [22]:
accelerator = Accelerator()
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)

In [32]:
def create_html_diff(text1, text2, chunk_num):
    """Create HTML diff between two texts"""
    # Wrap text to make it more readable
    text1_lines = textwrap.wrap(text1, width=80)
    text2_lines = textwrap.wrap(text2, width=80)
    
    # Create diff
    diff = HtmlDiff(wrapcolumn=80)
    html = diff.make_file(
        text1_lines, 
        text2_lines,
        fromdesc=f"Original (Chunk {chunk_num})",
        todesc=f"Processed (Chunk {chunk_num})",
        context=True
    )
    
    return html

In [34]:
def process_chunk(text_chunk):
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]
    
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():  # Add this for efficiency
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=8126
        )

    diff_html = create_html_diff(text_chunk, processed_text, chunk_num)
    
    return tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()

In [35]:
INPUT_FILE = "./extracted_text.txt"  # Replace with your file path
CHUNK_SIZE = 1000
output_file = f"clean_{os.path.basename(INPUT_FILE)}"
diff_file = f"diff_{os.path.basename(INPUT_FILE)}.html"

KeyError: ' font-family'

In [30]:
with open(INPUT_FILE, 'r', encoding='utf-8') as file:
    text = file.read()

# Calculate number of chunks
num_chunks = (len(text) + CHUNK_SIZE - 1) // CHUNK_SIZE

# Cell 6: Process the file
# Create output file name
output_file = f"clean_{os.path.basename(INPUT_FILE)}"

# Process chunks and write to file
processed_text = ""

with open(output_file, 'w', encoding='utf-8') as out_file:
    for chunk_num in tqdm(range(num_chunks), desc="Processing chunks"):
        # Get chunk with overlap
        start_idx = chunk_num * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        
        chunk = text[start_idx:end_idx]
        
        # Process chunk and append to complete text
        processed_chunk = process_chunk(chunk)
        processed_text += processed_chunk + " "
        
        # Write chunk immediately to file
        out_file.write(processed_chunk)
        
        # Force flush the file to disk
        out_file.flush()

Processing chunks:   0%|          | 0/101 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
1
A Survey on Knowledge Distillation of Large
Language Models
Xiaohan Xu1, Ming Li2, Chongyang Tao3, Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu
ckcheng@cs.hku.hk jl0725@connect.hku.hk
Abstract —In the era of Large Language Models (LLMs), Knowledge Distillati...

PROCESSED TEXT:
...



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
ed knowledge to smaller models and its utility in model compression and self-
improvement. Our survey is meticulously structured around three foundational pillars: algorithm ,skill, and verticalization – providing
a comprehensive examination of KD mechanisms, the enhancement of specific cognitive abilities, and their practical implications
across diverse fields. Crucially, the survey navigates the intricate interplay between data augmentation (DA) and KD, illustrating how
DA emerges as a powerfu...

PROCESSED TEXT:
Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu
ckcheng@cs.hku.hk
...



...

INPUT TEXT:
ionales serve
as a means to retrieve relevant knowledge d, and the student
LM is subsequently fine-tuned using the rationales along-
side questions and knowledge. However, during inference,
only questions are available. To address this, the Reranker
is trained to mimic how the retriever scores passages with
the rationale by minimizing the KL divergence between
Retriever (d|r)andReranker (d|x). However, the integra-
tion of a fixed number of passages in language models,
without considering their ...

PROCESSED TEXT:
Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu
ckcheng@cs.hku.hk
is meticulously structured around three foundational pillars: algorithm, skill, and verticalization – providing a comprehensive examination of knowledge distillatio

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
n Retrieve . To distill
this critic data, GPT-4 is prompted to assess the need for
retrieval using few-shot demonstrations I, the task input
x, and output yto predict a reflection token ras follows:
p(r|I, x, y ).
4.2 Alignment
4.2.1 Thinking Pattern
Most existing methods mainly focus on directly aligning the
direct responses of the student models to the responses of
teacher models (Taori et al., 2023). Though effective, these
models might suffer the problems that they tend to learn to
imitate t...

PROCESSED TEXT:
Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu
ckcheng@cs.hku.hk
is meticulously structured around three foundational pillars: algorithm, skill, and verticalization – providing a comprehensive examination of knowledge distillatio

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
2022; Madaan et al., 2023; Saunders
et al., 2022), SelFee (Ye et al., 2023) proposes to train a
model that has been fine-tuned to continuously revise its
own answer until it provides a high-quality response in a
single inference. During training, it utilizes both the final
response and feedback chain as the fitting target. This pat-
tern, response with the revision process, shows a promising
performance gain. Following SelFee, Reflection-Tuning (Li
et al., 2023e, 2024d) also utilizes the reflect...

PROCESSED TEXT:
Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu
ckcheng@cs.hku.hk
is meticulously structured around three foundational pillars: algorithm, skill, and verticalization – providing a comprehensive examination of knowledge distillatio